In [ ]:
import logging
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType
import psycopg2
from psycopg2 import sql

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Securely retrieve credentials for external systems
aws_access_key = dbutils.secrets.get(scope="aws", key="access_key")
aws_secret_key = dbutils.secrets.get(scope="aws", key="secret_key")

# Define helper functions for external data access
def fetch_data_from_postgresql(query, connection_params):
    try:
        conn = psycopg2.connect(**connection_params)
        cursor = conn.cursor()
        cursor.execute(query)
        data = cursor.fetchall()
        cursor.close()
        conn.close()
        return data
    except Exception as e:
        logger.error(f"Error fetching data from PostgreSQL: {e}")
        raise

# Load data from Unity Catalog tables
hospital_stats_df = spark.table("catalog.healthcare.hospital_stats_north_america")
sales_associates_df = spark.table("catalog.sales.sales_associates_employment_details")
compensation_guidelines_df = spark.table("catalog.sales.compensation_guidelines")
hospital_sales_assignments_df = spark.table("catalog.sales.hospital_sales_assignments")
logistics_channels_df = spark.table("catalog.sales.logistics_channels")
growth_opportunities_df = spark.table("catalog.sales.growth_opportunities")
third_party_sales_trends_df = spark.table("catalog.sales.third_party_sales_trends")
historical_sales_df = spark.table("catalog.sales.historical_sales")
company_goals_df = spark.table("catalog.sales.company_goals")

# Transformation logic
try:
    # Join hospital statistics with sales assignments
    hospital_sales_df = hospital_stats_df.join(
        hospital_sales_assignments_df,
        hospital_stats_df.Hospital_ID == hospital_sales_assignments_df.Hospital_ID,
        "inner"
    ).select(
        hospital_stats_df.Hospital_ID.alias("Left_Hospital_ID"),
        hospital_stats_df.Hospital_Name.alias("Left_Hospital_Name"),
        hospital_sales_assignments_df.Director_Name.alias("Right_Director_Name"),
        hospital_sales_assignments_df.Manager_Name.alias("Right_Manager_Name"),
        hospital_sales_assignments_df.Associate_ID.alias("Right_Associate_ID"),
        hospital_sales_assignments_df.Associate_Name.alias("Right_Associate_Name")
    )

    # Join employment details with compensation guidelines
    associate_compensation_df = sales_associates_df.join(
        compensation_guidelines_df,
        sales_associates_df.Associate_ID == compensation_guidelines_df.Associate_ID,
        "inner"
    ).select(
        sales_associates_df.Associate_ID.alias("Left_Associate_ID"),
        sales_associates_df.Associate_Name.alias("Left_Associate_Name"),
        compensation_guidelines_df.Base_Salary.alias("Right_Base_Salary"),
        compensation_guidelines_df.Commission_Percentage.alias("Right_Commission_Percentage"),
        compensation_guidelines_df.Bonus.alias("Right_Bonus")
    )

    # Consolidate associate and hospital data
    consolidated_df = associate_compensation_df.join(
        hospital_sales_df,
        associate_compensation_df.Left_Associate_ID == hospital_sales_df.Right_Associate_ID,
        "inner"
    )

    # Calculate total compensation
    consolidated_df = consolidated_df.withColumn(
        "Compensation",
        F.col("Right_Base_Salary") + (F.col("Right_Commission_Percentage") / 100 * F.col("Right_Base_Salary")) + F.col("Right_Bonus")
    )

    # Select relevant fields
    selected_df = consolidated_df.select(
        "Left_Hospital_ID", "Right_Director_Name", "Right_Manager_Name", "Left_Associate_ID", "Left_Associate_Name",
        "Compensation", "Left_Hospital_Name"
    )

    # Join logistics channels with growth opportunities
    logistics_growth_df = logistics_channels_df.join(
        growth_opportunities_df,
        logistics_channels_df.Channel_ID == growth_opportunities_df.Channel_ID,
        "inner"
    ).select(
        logistics_channels_df.Channel_ID.alias("Left_Channel_ID"),
        logistics_channels_df.Channel_Type.alias("Left_Channel_Type"),
        logistics_channels_df.Hospital_ID.alias("Left_Hospital_ID"),
        logistics_channels_df.Growth_Opportunities.alias("Left_Growth_Opportunities"),
        growth_opportunities_df.Projected_Growth_Rate.alias("Right_Projected_Growth_Rate")
    )

    # Remove duplicates
    unique_logistics_growth_df = logistics_growth_df.dropDuplicates(["Left_Channel_ID", "Left_Channel_Type", "Left_Hospital_ID"])

    # Join compensation data with growth opportunities
    compensation_growth_df = selected_df.join(
        unique_logistics_growth_df,
        selected_df.Left_Hospital_ID == unique_logistics_growth_df.Left_Hospital_ID,
        "inner"
    )

    # Select relevant fields
    final_selected_df = compensation_growth_df.select(
        "Left_Hospital_ID", "Left_Channel_Type", "Left_Growth_Opportunities", "Right_Projected_Growth_Rate"
    )

    # Join historical sales with third-party sales trends
    sales_trends_df = historical_sales_df.join(
        third_party_sales_trends_df,
        historical_sales_df.Channel_Type == third_party_sales_trends_df.Channel_Type,
        "inner"
    ).select(
        historical_sales_df.Year.alias("Left_Year"),
        historical_sales_df.Channel_ID.alias("Left_Channel_ID"),
        historical_sales_df.Channel_Type.alias("Left_Channel_Type"),
        historical_sales_df.Sales_Revenue.alias("Left_Sales_Revenue"),
        historical_sales_df.Hospital_ID.alias("Left_Hospital_ID"),
        third_party_sales_trends_df.Market_Trend.alias("Right_Market_Trend"),
        third_party_sales_trends_df.Political_Impact.alias("Right_Political_Impact"),
        third_party_sales_trends_df.Economic_Impact.alias("Right_Economic_Impact")
    )

    # Remove duplicates
    unique_sales_trends_df = sales_trends_df.dropDuplicates(["Left_Year", "Left_Channel_Type", "Left_Sales_Revenue"])

    # Join unique sales data with growth opportunities
    sales_growth_df = unique_sales_trends_df.join(
        final_selected_df,
        unique_sales_trends_df.Left_Hospital_ID == final_selected_df.Left_Hospital_ID,
        "inner"
    )

    # Generate rows for target years
    target_years_df = spark.range(2023, 2027).withColumnRenamed("id", "Target Year")

    # Calculate projected sales growth rate
    sales_growth_df = sales_growth_df.crossJoin(target_years_df).withColumn(
        "Projected_Sales_Growth_Rate",
        F.when(F.col("Target Year") == 2024, F.col("Right_Projected_Growth_Rate") + (F.col("Right_Projected_Growth_Rate") / 100))
        .when(F.col("Target Year") == 2025, (F.col("Right_Projected_Growth_Rate") + (F.col("Right_Projected_Growth_Rate") / 100)) + (F.col("Right_Projected_Growth_Rate") / 100))
        .when(F.col("Target Year") == 2026, ((F.col("Right_Projected_Growth_Rate") + (F.col("Right_Projected_Growth_Rate") / 100)) + (F.col("Right_Projected_Growth_Rate") / 100)) + (F.col("Right_Projected_Growth_Rate") / 100))
        .otherwise(F.col("Right_Projected_Growth_Rate"))
    )

    # Calculate projected revenue
    sales_growth_df = sales_growth_df.withColumn(
        "Projected Revenue",
        F.when(F.col("Target Year") == 2024, F.col("Left_Sales_Revenue") * (F.col("Projected_Sales_Growth_Rate") / 100))
        .when(F.col("Target Year") == 2025, F.col("Left_Sales_Revenue") * (1 + F.col("Projected_Sales_Growth_Rate") / 100))
        .when(F.col("Target Year") == 2026, F.col("Left_Sales_Revenue") * (1 + F.col("Projected_Sales_Growth_Rate") / 100))
        .otherwise(F.col("Left_Sales_Revenue"))
    )

    # Filter records based on Target Year
    filtered_df = sales_growth_df.filter(F.col("Target Year") > 2023)

    # Select relevant fields for output
    output_df = filtered_df.select(
        "Left_Hospital_ID", "Left_Channel_Type", "Left_Sales_Revenue", "Right_Market_Trend", "Right_Political_Impact",
        "Right_Economic_Impact", "Target Year", "Projected_Sales_Growth_Rate", "Projected Revenue"
    )

    # Sort records by Target Year
    sorted_output_df = output_df.orderBy("Target Year")

    # Write to Unity Catalog target table
    spark.sql("DROP TABLE IF EXISTS catalog.sales.target_sales_report")
    sorted_output_df.write.format("delta").mode("overwrite").saveAsTable("catalog.sales.target_sales_report")

    logger.info("ETL process completed successfully.")

except Exception as e:
    logger.error(f"Error during ETL process: {e}")
    raise
